In [1]:
import glob2
import torch
from step1_network import CLIPPhi2Model, train_model, frange_cycle_linear
from step1_dataset import collate_fn, llavadataset
from torch.utils.data import random_split, DataLoader
import wandb
import torch.nn as nn
from transformers import AutoTokenizer
import pickle

In [2]:
with open("coco_dataset_pickle", "rb") as fp:   # Unpickling
    coco_unpickle = pickle.load(fp)

In [3]:
clip_model_name  = "openai/clip-vit-base-patch32"
phi_model_name   = "microsoft/phi-2"
train_batch_size = 2
val_batch_size   = 4
device     = 'cuda'
tokenizer  = AutoTokenizer.from_pretrained(phi_model_name, trust_remote_code=True)

# model
MModalGPT        = CLIPPhi2Model().to(device)
max_steps        = 100000
model_save_step  = 1000
model_val_step   = 1000
log_step         = 1000
max_token_filter = 35 # memory management restriction

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
# data loaders
train_dataloader = DataLoader(llavadataset(coco_unpickle, phi_model_name,clip_model_name,'train',tokenizer),
                  collate_fn=collate_fn, batch_size=train_batch_size, num_workers = 10, shuffle=True, pin_memory=True)

val_dataloader   = DataLoader(llavadataset(coco_unpickle, phi_model_name,clip_model_name,'val',tokenizer),
                  collate_fn=collate_fn, batch_size=val_batch_size, num_workers = 10, shuffle=True, pin_memory=True)

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, MModalGPT.parameters()), lr=1e-5)

Train size 532577 and validation size 59176
Train size 532577 and validation size 59176


In [ ]:
wandb.init(project="clip_phi2_project", name="step1_pretrain")
torch.set_float32_matmul_precision('medium')
train_model(MModalGPT, train_dataloader, val_dataloader, optimizer, device, max_steps,model_save_step,model_val_step,log_step,max_token_filter,tokenizer)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sanjanatule. Use `wandb login --relogin` to force relogin


Training started.
Step 0/100000: Avg Running Loss = 3.715071678161621
Batch skipped as captions too long.
Batch skipped as captions too long.
Saving Checkpoint
0 - Target captions:
 A table topped with wine glasses full of wine.<|endoftext|>  
0 - predicted_captions:
 A table with a variety of food and drinks on it a. a. a. a.<|endoftext|> 
1 - Target captions:
 A white computer monitor on a cluttered wooden table.  
1 - predicted_captions:
 A laptop on a desk with a desk and a desk. a.. a. a.<|endoftext|> 
2 - Target captions:
 a sleek white imac mouse made by Apple<|endoftext|><|endoftext|>  
2 - predicted_captions:
 A laptop on a table with a a a a a. a. a. a. a<|endoftext|> 
3 - Target captions:
 Remote and various clutter on sofa chair of mattress.<|endoftext|>  
3 - predicted_captions:
 A bed with a laptop on it a bed a. a. a. a. a.<|endoftext|> 
Step 1000/100000: Avg Running Loss = 4.658684540987014
Batch skipped as captions too long.
Batch skipped as captions too long.
Saving C